In [7]:
import os
import pandas as pd
import mysql.connector
import openpyxl
from openpyxl.worksheet.table import Table, TableStyleInfo

from openpyxl import load_workbook
from openpyxl.styles import Border, Side
from openpyxl.styles import NamedStyle
from openpyxl.styles import PatternFill

In [8]:
# username = 'root'
# password = '000000'
# hostname = '127.0.0.1'
# port = 3306
# database = 'vendiman'

# MySQL database connection settings

# Main df

db_config = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': '000000',
    'database': 'vendiman'}


main_query= "SELECT * FROM uvm WHERE DATE_FORMAT(Date1, '%Y-%m') = '2024-05'; "

conn = mysql.connector.connect(**db_config)
main_df = pd.read_sql(main_query, conn)
conn.close()

C:\Users\Admin\AppData\Local\Temp\ipykernel_26272\1436073314.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  main_df = pd.read_sql(main_query, conn)


In [9]:
nic_product_names = pd.DataFrame(pd.read_excel('nic_mrp.xlsx'))
vendekin = pd.read_excel('Vendekin_data.xlsx')

main_df['Date1'] = pd.to_datetime(main_df['Date1'])
main_df['Month'] = main_df['Date1'].dt.month
main_df['Year'] = main_df['Date1'].dt.year


df_without_discount = pd.merge(main_df, nic_product_names, on='Item', how='left')
df_without_discount['Amount'] = df_without_discount['OG MRP'].fillna(main_df['Amount'])
df_without_discount = df_without_discount.drop(['OG MRP', 'Brand' , 'current MRP'], axis=1)


df_UVM_condition1 = df_without_discount['Status1'] == 'SUCCESS'
df_UVM_condition2 = df_without_discount['Month'] == 5
df_UVM_success = df_without_discount[df_UVM_condition1 & df_UVM_condition2]


df_UVM_success['ClientLocationName'] = df_UVM_success['ClientLocationName'].replace('Kewal Kiran 460/7 Goregaon East Mumbai', 'Kewal Kiran 460-7 Goregaon East Mumbai')
df_UVM_success['MachineUID'] = df_UVM_success['MachineUID'].astype(str)


uvm_sumif_table = df_UVM_success.groupby('MachineUID').agg({'Amount': 'sum', 'OrderId': 'count'}).reset_index()
uvm_sumif_table.columns = ['UID', 'sales', 'vends']


vendekin['machineuid'] = vendekin['machineuid'].astype(str)
vendekin = vendekin.rename(columns={'machineuid': 'UID'})


vendekin_sumif_table = vendekin.groupby('UID').agg({'amountpaid': 'sum', 'qty': 'count'}).reset_index()
vendekin_sumif_table.columns = ['UID', 'sales', 'vends']




C:\Users\Admin\AppData\Local\Temp\ipykernel_26272\1216244448.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_UVM_success['ClientLocationName'] = df_UVM_success['ClientLocationName'].replace('Kewal Kiran 460/7 Goregaon East Mumbai', 'Kewal Kiran 460-7 Goregaon East Mumbai')
C:\Users\Admin\AppData\Local\Temp\ipykernel_26272\1216244448.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_UVM_success['MachineUID'] = df_UVM_success['MachineUID'].astype(str)


In [13]:
table = pd.concat([uvm_sumif_table, vendekin_sumif_table], axis=0)
table_final = table.groupby('UID').agg({'sales': 'sum', 'vends': 'sum'}).reset_index()

mm = pd.read_excel('MM.xlsx')

mm['UID'] = mm['UID'].astype(str)
mm_df = mm[['UID', 'Ops Loc' , 'Client Name' , 'Client Location Name' , 'Machinename']]


mm_df.rename(columns = {'Client Name':'ClientName', 'Client Location Name':'ClientLocationName'}, inplace = True)

merged_df = pd.merge(table_final, mm_df, on='UID', how='inner')
merged_df

# merged_df.to_excel('check.xlsx')

C:\Users\Admin\AppData\Local\Temp\ipykernel_26272\4098001062.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mm_df.rename(columns = {'Client Name':'ClientName', 'Client Location Name':'ClientLocationName'}, inplace = True)


,UID,sales,vends,Ops Loc,ClientName,ClientLocationName,Machinename
0,1012,11610,560,Noida,Tata Consultancy Services Limited (Sodexo),TCS Okaya Centre Sec 62 Noida,"TCS, 000, B5, Block B, Sector 62, Okaya Centre"
1,104,28936,1462,Chandigarh,Infosys Limited,Infosys Manimajra Chandigarh,"Infosys, 000, Block No.1, Rajiv Gandhi Tech Pa..."
2,1040,7895,339,Mumbai Zone 4,Tata Consultancy Services Limited,TCS Olympus Thane Mumbai,"TCS, 013, Olympus, Hiranandani Estate, Thane"
3,1041,10015,437,Mumbai Zone 4,Tata Consultancy Services Limited,TCS Olympus Thane Mumbai,"TCS, 012, A, Olympus, Hiranandani Estate, Thane"
4,1042,120995,5176,Gurgaon,Google Connect Services India Pvt Ltd (Gurgaon),GOC Service India Pvt. Ltd (Google Gurgaon),"Google, 005, Café, Tower C, Unitech Signature ..."
...,...,...,...,...,...,...,...
1378,985,127305,5229,Gurgaon,Google Connect Services India Pvt Ltd (Gurgaon),GOC Service India Pvt. Ltd (Google Gurgaon),"Google, 009, Tower C, Unitech Signature Tower"
1379,986,12060,494,Gurgaon,Google Connect Services India Pvt Ltd (Gurgaon),Google Connect Unitech Signature Tower-C Gurgaon,"Google, 004, Tower C, Unitech Signature Tower"
1380,987,16245,645,Gurgaon,Google Connect Services India Pvt Ltd (Gurgaon),Google Connect Unitech Signature Tower-C Gurgaon,"Google, 005,A, Café, Tower C, Unitech Signatur..."
1381,988,7780,342,Mumbai Zone 4,Tata Consultancy Services Limited,TCS Olympus Thane Mumbai,"TCS, 011, Olympus, Hiranandani Estate, Thane"


In [20]:
CLN_list = ['TCS Olympus Thane Mumbai','TCS Gift City Tower 2 Gandhinagar','Vodafone Yerwada Pune','Vodafone Idea Hinjewadi Phase 1 Pune','TCS Garima Park Plot No. 41 Gandhinagar','Celebi Delhi Import Bldg-II IGI Airport Delhi','LT Foods MVL I Park Sector 15 Gurgaon','TCS Olympus B SEZ Unit I Thane Mumbai','RBI OLDR Fort Mumbai','Samsung India Express Avenue Royapettah Chennai','Cyberabad Convention Novotel Hotel Hyderabad','Vedanta Sesa Corporate Panjim Goa','Vedanta Pig Iron Amona Marcela Goa','Vedanta PIP Plant Navelim  Goa','Emerson Information Quark City F3 Tower Mohali','Cisco Commerce BPTP Udyog Vihar Gurgaon','HCL Elcot Sholinganallur Chennai','HCL Technologies Tower 3 WTC Kharadi Pune','HCL Pheonix Infocity Bldg 01 Madhapur Hyderabad','HCL Pheonix Infocity Bldg 08 Madhapur Hyderabad','HCL Technologies C 4 block Manyata Hobli Bengaluru','HCL Technologies V-IT Park Orion Block Hyderabad','HCL Technologies Jigani Bommasandra Bengaluru','HCL Technologies SEZ Unit-V Gandipet Hyderabad','ANSR Global BCIT Tower 2 Bhartiya City Bengaluru','HCL Technologies Tower 7 Cybercity Magarpatta Pune','HCL Technologies Amberfort Hinjewadi Pune','HCL Technologies Qubix Embassy Zone Hinjewadi Pune','HCL Technologies Quadra 1 Hadapsar Pune','HCL Technologies Embassy Rajiv Gandhi Marunji Pune','Akamai Tech Agusta Embassy Golf Domlur Bengaluru','TRIL IT4 Intellion Dindoshi Malad Mumbai','ManpowerGroup Spectra Hiranandani Business Mumbai','Symphony Ltd Symphony House Bokadev Ahmedabad','3M ANSR Global SEZ HUB4 Karle Town Nagavara Bengaluru','Mundra Petrochem Adani CH4 Prahalad Ngr Ahmedabad','UIA India Tata Nagar Kodigehealli Bengaluru','Technosoft Engg Knowledge Gateway Wagle Thane','Nihilent Weikfield Pune','Protean eGov Tech Lower Parel Mumbai','Synergy Maritime Powai Mumbai','Reserve Bank Of India Gomati Nagar Lucknow','Apar Industries MIDC Admin Bldg Rabele Mumbai','Ajmera Reality and Infra Andheri Mumbai','Morgan Stanley Nirlon Knowledge Goregaon Mumbai','Morgan Stanley Athena Bldg Goregaon Mumbai','Morgan Stanley Campus 8B Outer Ring Road Bengaluru','Morgan Stanley Campus 6A Outer Ring Road Bengaluru','CoWrks India Ecoworld Outer Ring Road Bengaluru','CoWrks India Purva Premier Residency Rd Bengaluru','CoWrks India Birla Centurian Worli Mumbai','CoWrks India Prudential Powai Mumbai','OLA Electric Salarpuria Electronic City Bengaluru','CoWrks India RMZ Kadanchavadi Chennai','CoWrks India Skyview Madhapur Hyderabad','Asset Reconstruction Ind Ltd The Ruby Dadar Mumbai','KRBL Ltd C-32 Sector 62 Noida','L&T Defence  Plant engg dept NB1 Powai Mumbai','Palaksha University Sec 101 SAS Nagar Mohali','Lumina Datamatics SDF Andheri Mumbai','Torrent Pharma Kasarvadi Dadar Mumbai','BirlaSoft Kundalahalli Vill. Brookfield Bengaluru','ANSR Global Manyata Embassy Nagawara Bengaluru','Lithium Urban Tech Seetharampalya Hoodi Bengaluru','ATMECS Tech Northgate Park Venkatala Bengaluru','Fairfield by Marriott - Curti Benaulim Beach Goa','MASS Hospitals Greater Kailash Delhi','Allegis Service Okhiyampettai Thoraipakkam Chennai','Barracuda Networks Hosur Bengaluru','Mashreq Global ITPL Main Rd Bengaluru','BM Birla Heart National Library Kolkata','Allegis Service Yerwada Pune','Allegis Service Madhapur Hyderabad','Junomoneta Finsol Gift City Gandhinagar','LSEG (CBRE) Bennigana Halli Bengaluru','Sanghvi Vasant Kunj Delhi','MediaNet Andheri Mumbai','Runwal Residency Dombivli  Mumbai','Bengaluru International Airport Devanahalli','Runwal Apartments Sion Mumbai','HCL Elcot Sozhinganallur Chennai (NIC)','Tata Steel Downstream Sector V Kolkata','Cisco Commerce Mahatma Gandhi Road Bengaluru','Clicktech Prestige Sterling Sq Ashok Ngr Bengaluru','Adani Def Sys & Tech Mamidipally Village Hyderabad','Solus Hiranandani Estate Ghodbunder Thane','Reserve Bank Of India CBD Belapur Navi Mumbai','LSEG (CBRE) Divyasree Technopolis Hobli Bangalore','Chanakya University Devanahalli Bengaluru','HCL DivyaSree Solitaire Ranga Reddy Hyderabad','Molex India - Laxmi Caterers Devanahalli Bangalore','Vitech Systems Asia 12D Madhapur Hyderabad','Wipro Tower 5 Sector 5 Lake city Kolkata','Himadri Chems - AIPL Navgavra Kolkata','Oracle Solution - JLL Nagavara Bengaluru','NRM Consultants DLF Cybercity Gurgaon','Staypocket H01A Sector 63 Noida','Spectraforce Atrium 204 SAS Nagar Chandigarh','Biocon Biologics Semicon Electronic City Bangalore','Biocon Biologics PB Soft Electronic City Bengaluru','Indian Spinal Injuries OPD Vasant Kunj Delhi','Honeywell Adarsh Palm Retreat Bellandur Bangalore','Honeywell RMZ Ecoworld Bellandur Benagaluru','HOM India Sector 136 Noida','Deloitte Tax Services Valence Outer Ring Bangalore','Datamark BPO Thane One Dil Complex Thane Mumbai']

for CLN_filter in CLN_list:
    filtered_df1 = df_UVM_success[df_UVM_success['ClientLocationName'] == CLN_filter]
    filtered_df2 = vendekin[vendekin['ClientLocationName'] == CLN_filter]
    filtered_df3 = merged_df[merged_df['ClientLocationName'] == CLN_filter]

    with pd.ExcelWriter(f'{CLN_filter}.xlsx', engine='openpyxl') as writer:
        filtered_df1.to_excel(writer, sheet_name='UVM data', index=False)

    with pd.ExcelWriter(f'{CLN_filter}.xlsx', engine='openpyxl', mode='a') as writer:
        filtered_df2.to_excel(writer, sheet_name='Vendekin Data', index=False)

    with pd.ExcelWriter(f'{CLN_filter}.xlsx', engine='openpyxl', mode='a') as writer:
        filtered_df3.to_excel(writer, sheet_name='Data', index=False)

In [21]:
import openpyxl
from openpyxl.styles import Border, Side, PatternFill

# Replace 'your_file.xlsx' with the actual file name
for i in CLN_list:
    file_path = f'{i}.xlsx'
    sheet_name = 'Data'

    # Load the workbook
    workbook = openpyxl.load_workbook(file_path)

    # Access the 'Data' sheet
    sheet = workbook[sheet_name]

    # Autofit all columns
    for column in sheet.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        sheet.column_dimensions[column[0].column_letter].width = adjusted_width

    # Add a black border to the entire table
    border = Border(left=Side(border_style='thin', color='000000'),
                    right=Side(border_style='thin', color='000000'),
                    top=Side(border_style='thin', color='000000'),
                    bottom=Side(border_style='thin', color='000000'))

    for row in sheet.iter_rows():
        for cell in row:
            cell.border = border

    # Add hex color ('DA9694') to cells A1 to G1
    fill = PatternFill(start_color='DA9694', end_color='DA9694', fill_type='solid')

    for col_num in range(1, 8):  # Assuming A to G
        sheet.cell(row=1, column=col_num).fill = fill

    # Save the modified workbook
    workbook.save(file_path)


In [22]:
import win32com.client
import os
import pandas as pd

def send_email_with_attachment(subject, body, to_emails, cc_emails, attachment_path):
    outlook = win32com.client.Dispatch("Outlook.Application")
    mail = outlook.CreateItem(0)
    
    mail.Subject = subject
    mail.Body = body
    
    # Add 'To' recipients
    if to_emails:
        mail.To = ";".join(to_emails)
    
    # Add 'CC' recipients
    if cc_emails:
        mail.CC = ";".join(cc_emails)
    
    # Attach the Excel file
    if os.path.isfile(attachment_path):
        mail.Attachments.Add(attachment_path)
    else:
        print(f"Attachment file '{attachment_path}' not found.")
        return
    
    # Send the email
    mail.Send()

def send_emails_from_excel(file_path):
    # Read Excel file into a DataFrame
    df = pd.read_excel(file_path)

    # Iterate through rows
    for index, row in df.iterrows():
        subject = row['Subject']
        body = row['Body']
        
        # Split email addresses and remove leading/trailing whitespaces
        to_emails = [email.strip() for email in row['To'].split(',')]
        cc_emails = [email.strip() for email in row['CC'].split(',')]
        
        attachment_path = row['Attachment']

        send_email_with_attachment(subject, body, to_emails, cc_emails, attachment_path)

# Example usage
excel_file_path = r'C:\Users\Admin\OneDrive - VENDIMAN PVT LTD\Amanda\Sales report - Client\May 2024\emailers.xlsx'
send_emails_from_excel(excel_file_path)
